In [10]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [11]:
spark = SparkSession.builder.appName("solution_ex_19").getOrCreate()

In [12]:
inputPath = './data'
outputPath = './output_ex_19/df_sol'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [13]:
# Create a DataFrame from persons.csv
dfPersons = spark.read.load(inputPath,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)
dfPersons.show()

+-----+-------+---+
| name|surname|age|
+-----+-------+---+
|Paolo|  Garza| 42|
| Luca| Boccia| 41|
|Maura|Bianchi| 16|
+-----+-------+---+



In [14]:
def agecat(age):
    return "["+str((age//10)*10)+"-"+str((age//10)*10+9)+"]"


spark.udf.register("AgeCategory", agecat)

# version 2
# spark.udf.register("range_age", lambda age: '['+str((age//10)*10)+'-'+str((age//10)*10+9)+']')

22/06/03 16:58:24 WARN SimpleFunctionRegistry: The function agecategory replaced a previously registered function.


<function __main__.agecat(age)>

In [15]:
profilesDiscretizedAge = dfPersons.selectExpr("name", "surname", "AgeCategory(age) as rangeage")
profilesDiscretizedAge.show()

+-----+-------+--------+
| name|surname|rangeage|
+-----+-------+--------+
|Paolo|  Garza| [40-49]|
| Luca| Boccia| [40-49]|
|Maura|Bianchi| [10-19]|
+-----+-------+--------+



In [16]:
profilesDiscretizedAge.write.csv(outputPath)